In [5]:
import pandas as pd
import numpy as np
import xarray as xr

import os
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
from pyaldata import *

In [6]:
# Load data
path = "."
for (col,filename) in enumerate(os.listdir(path)):
    if filename.endswith('.mat'):
        fname = os.path.join(path, filename)
        #print(fname[:-4])
        
        # load TrialData .mat file into a DataFrame
        df = mat2dataframe(fname, shift_idx_fields=True)
        
        # combine time bins into longer ones
        td = combine_time_bins(df, 3)
        
        # Remove low-firing neurons
        td = remove_low_firing_neurons(td, "M1_spikes",  3)
        td = remove_low_firing_neurons(td, "PMd_spikes", 3)
        
        # Smooting the signal
        td = smooth_signals(td, ["M1_spikes", "PMd_spikes"], std=0.05)
        
        # Combine M1 and PMd
        td = merge_signals(td, ["M1_spikes", "PMd_spikes"], "both_spikes")
        
        N = td.shape[0]

        for i in range(N):


            # get position
            pos = td.pos[i]


            # get velocities for this trial
            vel = td.vel[i]


            # get M1 spikes
            M1 = td.M1_spikes[i]

            # get PMd_spikes
            PMd = td.PMd_spikes[i]

            # get both
            both_spikes = td.both_spikes[i]


            if (i == 0):
                grouped_pos = pos
                grouped_vel = vel

                grouped_M1 = M1
                grouped_PMd = PMd
                grouped_both_spikes = both_spikes
            else:
                grouped_pos = np.append(grouped_pos, pos, axis=0)
                grouped_vel = np.append(grouped_vel, vel, axis=0)
                grouped_M1 = np.append(grouped_M1, M1, axis=0)
                grouped_PMd = np.append(grouped_PMd, PMd, axis=0)
                grouped_both_spikes = np.append(grouped_both_spikes, both_spikes, axis=0);

        # Save the data in right format
        import pickle

        data_folder='data/' #FOLDER YOU WANT TO SAVE THE DATA TO

        with open(fname[:-4]+'.pickle','wb') as f:
            pickle.dump([grouped_M1, grouped_PMd, grouped_both_spikes, grouped_pos, grouped_vel],f)


        #fname = os.path.join(data_dir, "RS_ind_1.mat")